In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.regularizers import l2

In [2]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(tf.config.experimental.get_memory_growth(gpu))

True


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
def read_annotation_file(annotation_file_path):
    with open(annotation_file_path, 'r') as file:
        line = file.readline().strip()
        values = line.split()
        num_points = int(values[0])
        annotation_data = [int(value) for value in values[1:]]
        return num_points, annotation_data

# Lists to store image data and annotation data
images_and_annotations = []

# Path to the directory containing the images and .cat files
folder_path = r'C:\Users\haris\test-installation\Data\Face Mask\Cats'

# Lists to store paths of .jpg files and .cat files
image_files_path = []
annotation_files_path = []

for root, dirs, files in os.walk(folder_path):
    # Loop through all the files in the folder
    for file in files:
        # Check if the file has a .jpg extension
        if file.lower().endswith(('.jpg', '.jpeg', 'png')):
            image_path = os.path.join(root, file)
            image_files_path.append(image_path)

        # Check if the file has a .cat extension
        if file.lower().endswith('.cat'):
            annotation_path = os.path.join(root, file)
            annotation_files_path.append(annotation_path)
            
image_size = (224, 224)
# Read annotation data for each image
for annotation_file_path in annotation_files_path:
    num_points, annotation_data = read_annotation_file(annotation_file_path)

    # Assuming images are stored in RGB format
    image = cv2.imread(annotation_file_path[:-4], cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    # Resize images to a common size (e.g., 224x224)
    resized_image = cv2.resize(image, image_size)

    # # Calculate the scaling factor for annotation coordinates
    scale_y = resized_image.shape[0] / image.shape[0]
    scale_x = resized_image.shape[1] / image.shape[1]

    # Apply scaling to the annotation data for both x and y coordinates together
    resized_annotation_data = [int(value * scale_x) if i % 2 == 0 else int(value * scale_y) for i, value in enumerate(annotation_data)]
    images_and_annotations.append((resized_image, resized_annotation_data))
# Lists to store resized image data and annotation data
resized_images = []
resized_annotations = []

# Iterate through the images_and_annotations list
for resized_image, resized_annotation_data in images_and_annotations:
    resized_images.append(resized_image)
    resized_annotations.append(resized_annotation_data)

# Convert lists to numpy arrays
resized_images = np.array(resized_images)
resized_annotations = np.array(resized_annotations)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(resized_images, resized_annotations, test_size=0.1, random_state=42)


In [6]:
# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the pre-trained model with reduced dense layer units and L2 regularization
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)  # Reduce units to 256 with L2 regularization
predictions = Dense(18, activation='linear')(x)  # Output layer with 18 units for facial landmark points

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with a reduced batch size
model.fit(x_train, y_train, epochs=10, batch_size=4, validation_data=(x_test, y_test))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

: 